## Imports

In [1]:
from collections import OrderedDict
from bs4 import BeautifulSoup
from fuzzywuzzy import process
import altair as alt
import pandas as pd

from selenium.common.exceptions import TimeoutException
from selenium.webdriver import Chrome, ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait

# Scraping w/ Selenium 

In [2]:
zack_url = "https://scholar.google.com/citations?user=X7FY3wUAAAAJ&hl=en&oi=ao"
hinton_url = "https://scholar.google.com/citations?user=JicYPdAAAAAJ&hl=en&oi=ao"
malcolm_url = "https://scholar.google.de/citations?user=bcO-7KwAAAAJ&hl=en&oi=ao"
tarnas_url = "https://scholar.google.com/citations?hl=en&user=H9AJzHMAAAAJ&view_op=list_works),"
allegra_url = "https://scholar.google.com/citations?user=CfnwDC4AAAAJ&hl=en"
kessler_url = "https://scholar.google.com/citations?user=EicYvbwAAAAJ&hl=en&inst=5746887945952177237&oi=ao)"
foucault_url = "https://scholar.google.com/citations?user=AKqYlxMAAAAJ&hl=en&inst=5746887945952177237&oi=ao)"
bourdieu_url = "https://scholar.google.com/citations?user=d_lp40IAAAAJ&hl=en&inst=5746887945952177237&oi=ao),"

In [44]:
options = ChromeOptions()
options.headless = True
driver = Chrome(options=options)

In [45]:
button_xpath = "/html/body/div/div[13]/div[2]/div/div[4]/form/div[2]/div/button"

In [46]:
%%time
base_url = malcolm_url
# base_url = zack_url
# base_url = hinton_url
# base_url = tarnas_url
# base_url = allegra_url
# base_url = kessler_url
# base_url = foucault_url
# base_url = bourdieu_url
driver.get(base_url)
count = 0

while True:  
    try:
        button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.XPATH, button_xpath)))
        button.click()
        count += 1
        print(f"click number: {count}")
        
    except TimeoutException:
        print('not clickable')
        break
        
html = driver.page_source
base_page = BeautifulSoup(html, "lxml")
driver.close()

links = []
for td in base_page.find_all("td", attrs={"class": "gsc_a_t"}):
    link = td.find("a").get("data-href")
    full_link = base_url.split("citations?")[0] + link
    links.append(full_link)
print("number of publication links found: ", len(links))

not clickable
number of publication links found:  16
Wall time: 7.93 s


In [47]:
type(base_page)

bs4.BeautifulSoup

# Get data w/ BeautifulSoup

### Get author name

In [48]:
page_title = base_page.find('title').string
author_name = page_title.split(' - ')[0]
author_name

'Malcolm S.W. Hodgskiss'

### Get current role

In [49]:
temp = []
for x in base_page.find_all("div", attrs={"class": "gsc_prf_il"}):
    temp.append(x.text)
    
temp[0]

'Ph.D | Stanford University'

### Exclude duplicate publications

In [103]:
pubs_data = []
for tr in base_page.find_all("tr", attrs={"class": "gsc_a_tr"}):
    # check if it contains an attribute specific to duplicates
    if tr.find_all("a")[1].has_attr("data-eud"):
        continue
    
    else:
        td1, td2, td3 = tr.find_all("td")
        
        authors = td1.find("div").contents
        authors = authors[0].split(", ")
        authors = [name for name in authors if name!="..."]
        
        citations = td2.find("a").contents
        if len(citations) < 1:
            citations = None
        else:
            citations = int(citations[0])
            
        year = td3.find("span").contents    
        if len(year) < 1:
            year = None
        else:
            year = int(year[0])
        
        data = {
            "authors": authors, 
            "citations": citations, 
            "year": year
        }
        pubs_data.append(data)
    
print(len(pubs_data))

16


In [104]:
pubs_data[:2]

[{'authors': ['DB Cole',
   'CT Reinhard',
   'X Wang',
   'B Gueguen',
   'GP Halverson',
   'T Gibson'],
  'citations': 137,
  'year': 2016},
 {'authors': ['TM Gibson',
   'PM Shih',
   'VM Cumming',
   'WW Fischer',
   'PW Crockford'],
  'citations': 64,
  'year': 2018}]

### Get total citations

In [80]:
count = 0
for pub in pubs_data:
    if pub["citations"]:
        count += pub["citations"]
    
print(count)

300


### Get citation counts for each paper

There are missing values from the years list because sometimes that section is blank on the website, but the year is listed in the title of the paper section instead.

Also, it seems there are duplicate publications listed on some Google Scholar profiles. For example on Geoffrey Hinton's, "Layer Normalization" is listed three different times. Same authors, same journal (arXiv), but different citations counts.

Google is not taking this into account in there h-index calculations b/c I reproduced theirs by including duplicates.

# Format data by year(to-do), co-author pos, & h-index

### Get co-author positions

In [81]:
# scary one-liner to convert int to ordinal representation (ex: 2 --> "2nd")
# https://stackoverflow.com/questions/9647202/ordinal-numbers-replacement
# ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n/10%10!=1)*(n%10<4)*n%10::4])

In [100]:
print("author name: ", author_name)

def get_author_positions_lis(auth_name, auth_lists):
    ordinal = lambda n: "%d%s" % (n,"tsnrhtdd"[(n/10%10!=1)*(n%10<4)*n%10::4])
    author_positions_lis = []

    for names in auth_lists:
#         print(names)
        try: 
            matches = process.extract(auth_name, names, limit=2)
#             print(matches)
            if matches:
                # make sure they're not too far off origianl name
                # cutoff score: 75
                if matches[0][1] > 74:
                    
                    if len(matches) == 2:
                        # if one's better, take that one
                        if matches[0][1] > matches[1][1]:
                            match = matches[0][0]
                
                        else:
                            full_name_lis = auth_name.split()
                            last_name = full_name_lis.pop()
                            initials = "".join([name[0] for name in full_name_lis])
                            shortened_auth_name = initials + " " + last_name
                            match = process.extractOne(shortened_auth_name, [name[0] for name in matches])[0]
                    else:
                        match = matches[0][0]
                else:
                    match = None
                    
        except TypeError: 
            match = None
            print('no match')
            
        if match:
            for i, author in enumerate(names):
                if author == match:

                    if i == len(names)-1 and i > 2:
                        author_positions_lis.append('last')
                    elif i > 4:
                        author_positions_lis.append('≥ 6th')
                    else:
                        author_positions_lis.append(ordinal(i+1))
                    break
        else:
            if len(names) > 4:
                author_positions_lis.append('≥ 6th')
            else:
                author_positions_lis.append(ordinal(len(names)))
            
    return author_positions_lis

author_positions_lis = get_author_positions_lis(author_name, [pub["authors"] for pub in pubs_data])

print(author_positions_lis[:5])
print(len(author_positions_lis))

author name:  Malcolm S.W. Hodgskiss
['≥ 6th', '≥ 6th', '4th', '2nd', '4th']
16


In [83]:
set(author_positions_lis)

{'1st', '2nd', '4th', '≥ 6th'}

### Group citations by co-author position

In [85]:
def get_pos_dfs(pos_lis, num_lis):
    citations_positions_df = pd.DataFrame(list(zip(pos_lis, num_lis)), columns =['positions', 'citations']) 
    return dict(tuple(citations_positions_df.groupby('positions')))

citations_lis = [pub["citations"] for pub in pubs_data]
dfs = get_pos_dfs(author_positions_lis, citations_lis)
dfs.keys()

dict_keys(['1st', '2nd', '4th', '≥ 6th'])

In [86]:
dfs['1st'].head()

,positions,citations
5,1st,9.0
7,1st,8.0
9,1st,4.0
12,1st,NaN
13,1st,NaN


### Get h-indexes by co-author position

In [87]:
def get_hindexes_dict(dataframes):
    hindexes_dict = {}
    
    for k, df in dataframes.items():
        df.sort_values('citations')
        df.index += 1
        df = df.reset_index()
        df = df.query('citations >= index')
        # checking if there are no citations
        if df.shape[0] <= 0:
            hindexes_dict[k] = 0
        else:
            hindexes_dict[k] = df.shape[0]
        
    return hindexes_dict
        
hindexes_d = get_hindexes_dict(dfs)

In [88]:
# excluding co-author position w/ h-index of zero
hindexes_d

{'1st': 2, '2nd': 1, '4th': 3, '≥ 6th': 2}

In [89]:
sum(hindexes_d.values())

8

In [90]:
# Regular h-index (same as Google's)
temp_df = pd.DataFrame({"citations": citations_lis})
temp_df.sort_values('citations')
temp_df.index += 1
temp_df = temp_df.reset_index()
temp_df = temp_df.query('citations >= index')
temp_df.shape[0]

8

In [91]:
pd.DataFrame({'position': list(hindexes_d.keys()), 'h-index': list(hindexes_d.values())}).set_index('position')

,h-index
position,
1st,2
2nd,1
4th,3
≥ 6th,2


# Format data for plotting

In [92]:
def get_counts_dicts(pos_lis, num_lis):
    d1 = {}
    d2 = {}

    for position, num in zip(pos_lis, num_lis):
        if not num:
            continue
        else: 
            if position in d1:
                d1[position] += 1
                d2[position] += num
            else:
                d1[position] = 1
                d2[position] = num
    return d1, d2
        
author_positions, citations_by_author_position = get_counts_dicts(author_positions_lis, citations_lis)

print(f'author_positions: {author_positions}')
print(f'citations_by_author_position: {citations_by_author_position}')

author_positions: {'≥ 6th': 3, '4th': 5, '2nd': 1, '1st': 3}
citations_by_author_position: {'≥ 6th': 204, '4th': 56, '2nd': 19, '1st': 21}


In [93]:
author_positions = OrderedDict(sorted(author_positions.items()))
citations = OrderedDict(sorted(citations_by_author_position.items()))

In [94]:
if "last" in author_positions:
    author_positions.move_to_end('last')
if "last" in citations:
    citations.move_to_end('last')

In [95]:
citations

OrderedDict([('1st', 21), ('2nd', 19), ('4th', 56), ('≥ 6th', 204)])

In [96]:
author_positions

OrderedDict([('1st', 3), ('2nd', 1), ('4th', 5), ('≥ 6th', 3)])

In [97]:
lis1 = list(citations.keys())
lis2 = list(citations.values())

In [98]:
df = pd.DataFrame({
                "positions": lis1,
                "citations": lis2
            })
percents_df = df.copy()
percents_df.citations = (100 * df.citations / df.citations.sum()).round(0)
percents_df.citations = percents_df.citations / 100
# percents_df = percents_df.fillna(0)
percents_df

,positions,citations
0,1st,0.07
1,2nd,0.06
2,4th,0.19
3,≥ 6th,0.68


# Plot w/ Altair

In [99]:
def bar_chart(df):
    return alt.Chart(df).mark_bar().encode(
        alt.X('citations', axis=alt.Axis(format='%')),
        alt.Y('positions', sort=None),
        color=alt.Color(
            "positions", 
            scale=alt.Scale(scheme="greenblue"), 
            legend=None
            )
        ).properties(
            title='portion of total citations by co-author position'
        ).configure_axisX(
            labelAngle=0
        ).configure_view(
            strokeWidth=0
        )


bar_chart(percents_df).display()

alt.Chart(...)